# Texture-based separation to refine building meshes 

- Use Texture segmentation to separate the different materials in the meshes
- Detect the edges on the texture
- Convert the detected edges to hough lines
- project the uv coordinates of the mesh onto the plane
- Cut the triangles with the detected lines
- Define the new faces with the edge boundaries


In [ ]:
%load_ext autoreload
%autoreload 2

from context import segmentationtools
import cv2
import open3d as o3d
import math
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#import the 3D mesh
meshPath = "../../../localfiles/DemoHouse.obj"
mesh = o3d.io.read_triangle_mesh(meshPath, True)

## Texture Segmentation

- We want to group textures and reduce their detail for better edge detection
- use Factorisation based Texture segmentation


In [ ]:
import segmentationtools.fseg as fseg

texture = np.asarray(mesh.textures[0])
grayTexture = cv2.resize(cv2.cvtColor(texture, cv2.COLOR_BGR2GRAY), (512, 512))

# define filter bank and apply to image. for color images, convert rgb to grey scale and then apply filter bank
filter_list = [('log', .5, [3, 3]), ('log', 1, [5, 5]),
                ('gabor', 1.5, 0), ('gabor', 1.5, math.pi/2), ('gabor', 1.5, math.pi/4), ('gabor', 1.5, -math.pi/4),
                ('gabor', 2.5, 0), ('gabor', 2.5, math.pi/2), ('gabor', 2.5, math.pi/4), ('gabor', 2.5, -math.pi/4)
                ]
filter_out = fseg.image_filtering(grayTexture, filter_list=filter_list)
# include original image as one band
Ig = np.concatenate((np.float32(grayTexture.reshape((grayTexture.shape[0], grayTexture.shape[1], 1))), filter_out), axis=2)
# run segmentation. try different window size, with and without nonneg constraints
segmentedTexture = fseg.Fseg(Ig, ws=10, segn=0, omega=.2, nonneg_constraint=False)

In [ ]:
# show results
fig, ax = plt.subplots(ncols=2, sharex=True, sharey=True, figsize=(12, 6))
ax[0].imshow(grayTexture, cmap='gray')
ax[1].imshow(segmentedTexture)
plt.tight_layout()
plt.show()

In [ ]:

a = segmentedTexture
segmentedMappedTexture = np.round(np.interp(a, (a.min(), a.max()), (0, 255))).astype(np.uint8)


## Edge Detection

- The patched image is pefect to detect the boundaries of the textures
- Use canny or HED edge detection
- Find the hough lines to get straight lines in the dtected edges

In [ ]:
# perform Canny edge tedection to find sharp gradients
dst = cv2.Canny(segmentedMappedTexture, 100, 300, None, 3)
dstp = cv2.cvtColor(dst, cv2.COLOR_GRAY2BGR)

segmentationtools.show_img(dstp)

In [ ]:
dstp2 = cv2.cvtColor(dst, cv2.COLOR_GRAY2BGR)
linesP = cv2.HoughLinesP(dst, 1, np.pi / 360, 30, None, 0, 0)

if linesP is not None:
    for i in range(0, len(linesP)):
        l = linesP[i][0]
        cv2.line(dstp2, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv2.LINE_AA)

segmentationtools.show_img(dstp2)

In [ ]:
# Lets apply Standard HoughLine transform to detect lines
dstp2 = cv2.cvtColor(dst, cv2.COLOR_GRAY2BGR)
lines = cv2.HoughLines(dst, 2, np.pi / 120, 150, None, 0, 0)
print(lines)
# Below we will display the result by drawing lines
if lines is not None:
    for i in range(0, len(lines)):
        rho = lines[i][0][0]
        theta = lines[i][0][1]
        a = math.cos(theta)
        b = math.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = (int(x0 + 1000 * (-b)), int(y0 + 1000 * (a)))
        pt2 = (int(x0 - 1000 * (-b)), int(y0 - 1000 * (a)))
        cv2.line(dstp2, pt1, pt2, (0, 0, 255), 3, cv2.LINE_AA)

segmentationtools.show_img(dstp2)

## Face Slicing

- slice the triangles that are in the path of the new lines
- check for continuity in adjacent faces to continue the line

In [ ]:
# Plot the triangles on the uv plane
# Scale the uv coordinates with the image dimensions
uvPoints = np.asarray(mesh.triangle_uvs) * np.array([segmentedMappedTexture.shape[1],segmentedMappedTexture.shape[0]])
print("UV coordinates: \n", len(uvPoints))
print("nr of vertices: \n", len(mesh.vertices))
plt.imshow(segmentedMappedTexture)

for i in range((int)(len(uvPoints)/3)):
    plt.fill(*zip(*uvPoints[3*i:3*i+3]), facecolor='none', edgecolor='orangered', linewidth=1)
plt.title('UV Layout')
plt.show()

In [ ]:
import segmentationtools as st
points = []

lineNr = 0

for i in range((int)(len(uvPoints)/3)):
    newPoints = st.find_triangle_intersection(uvPoints[3*i:3*i+3], st.get_edge_points(lines[lineNr], segmentedMappedTexture.shape[1], segmentedMappedTexture.shape[0]))
    if (newPoints is not None): 
        points.append(newPoints)

points = np.array(points).reshape((-1,2)) #reshape to a nx2 array
print("The resulting intersection points are: \n", points)

rho = lines[lineNr][0][0]
theta = lines[lineNr][0][1]
a = math.cos(theta)
b = math.sin(theta)
x0 = a * rho
y0 = b * rho
pt1 = (int(x0 + 1000 * (-b)), int(y0 + 1000 * (a)))
pt2 = (int(x0 - 1000 * (-b)), int(y0 - 1000 * (a)))
cv2.line(segmentedMappedTexture, pt1, pt2, (0, 0, 255), 3, cv2.LINE_AA)

plt.imshow(segmentedMappedTexture)
plt.scatter(*zip(*points))
plt.title('my picture')
plt.show()


In [ ]:
# Perform the triangle slicing algorithm for each triangle seperatly
import segmentationtools
uv_coords = np.asarray(mesh.triangle_uvs) * np.array([segmentedMappedTexture.shape[1],segmentedMappedTexture.shape[0]])
verts = np.asarray(mesh.vertices)
bounds = [segmentedMappedTexture.shape[1], segmentedMappedTexture.shape[0]]
edgePoints = st.get_edge_points(lines[lineNr],bounds[0], bounds[1])

new_vertices = []
new_uvs = []
new_triangles = []

for triangle in mesh.triangles:
    print(triangle)
    points_uv = np.array([uv_coords[triangle[0]],uv_coords[triangle[1]],uv_coords[triangle[2]]])
    points_3d = np.array([verts[triangle[0]],verts[triangle[1]],verts[triangle[2]]])
    #print(points_uv)
    #print(points_uv.shape, edgePoints.shape)
    newTriangles,newTriangles_3d  = st.cut_triangle(points_uv, edgePoints, points_3d)
    if (len(newTriangles) == 3): 
        print("Got an intersection")
        print(newTriangles,newTriangles_3d)



In [ ]:
# I have a list of new triangles, both with uv coordinates and 3d coordinates
# To put them back in the mesh in need:
# Get a list of the new points and uv coordinates
# Add them to the existing mesh list
# Remove the old triangles and replace them with new indexes

## Object Segmentation

- use the texture blobs and adjacent faces region growing to segment the different objects

In [ ]:
import segmentationtools as st
mesh.textures = [o3d.geometry.Image(cv2.cvtColor(segmentedMappedTexture, cv2.COLOR_GRAY2RGB))]
print(mesh.textures)


In [ ]:
st.show_geometries([mesh])

In [ ]:
# split the mesh based on the texture color and adjacency

# open3d adjacency list to get neighbouring verteces